In [212]:
import imblearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## ROUND 1

In [213]:
churnData = pd.read_csv("DATA_Customer-Churn.csv")

In [214]:
churnData

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [215]:
churnData.info()
# print(churnData.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [216]:
churnData.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [217]:
# s = pd.Series(["TotalCharges"])
# pd.to_numeric(s, errors='coerce')

In [218]:
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors='coerce')

In [219]:
churnData.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [220]:
print(churnData.isnull().sum())

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


In [221]:
churnData.isnull().any(axis=1).value_counts()

False    7032
True       11
dtype: int64

In [222]:
churnData = churnData.fillna(0)
churnData.isnull().any(axis=1).value_counts()

False    7043
dtype: int64

In [223]:
churnData.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304
std,0.368612,24.559481,30.090047,2266.794470
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


In [224]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

## SELECTING FEATURES

In [225]:
selected_features = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
churnData[selected_features]

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65
...,...,...,...,...
7038,24,0,84.80,1990.50
7039,72,0,103.20,7362.90
7040,11,0,29.60,346.45
7041,4,1,74.40,306.60


In [226]:
X = churnData[selected_features]
y = churnData['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Scaling

In [227]:
# with MinMaxScaler
MMscaler = MinMaxScaler()
X_train_scaled = MMscaler.fit_transform(X_train)
X_test_scaled = MMscaler.transform(X_test)

# with StandardScaler
SSscaler = StandardScaler()
X_train_scaled = SSscaler.fit_transform(X_train)
X_test_scaled = SSscaler.transform(X_test)

In [228]:
MMscaler

MinMaxScaler()

In [229]:
SSscaler

StandardScaler()

## ROUND 2

In [230]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

logistic_model = LogisticRegression()
knn_model = KNeighborsClassifier()
tree_model = DecisionTreeClassifier()

logistic_model.fit(X_train_scaled, y_train)
knn_model.fit(X_train_scaled, y_train)
tree_model.fit(X_train_scaled, y_train)

# Make predictions on train and test sets
logistic_train_pred = logistic_model.predict(X_train_scaled)
logistic_test_pred = logistic_model.predict(X_test_scaled)

knn_train_pred = knn_model.predict(X_train_scaled)
knn_test_pred = knn_model.predict(X_test_scaled)

tree_train_pred = tree_model.predict(X_train_scaled)
tree_test_pred = tree_model.predict(X_test_scaled)

# Evaluation metrics for logistic regression
logistic_train_accuracy = accuracy_score(y_train, logistic_train_pred)
logistic_train_precision = precision_score(y_train, logistic_train_pred, pos_label='Yes')
logistic_train_recall = recall_score(y_train, logistic_train_pred, pos_label='Yes')

logistic_test_accuracy = accuracy_score(y_test, logistic_test_pred)
logistic_test_precision = precision_score(y_test, logistic_test_pred, pos_label='Yes')
logistic_test_recall = recall_score(y_test, logistic_test_pred, pos_label='Yes')


# Evaluation metrics for KNN classifier
knn_train_accuracy = accuracy_score(y_train, knn_train_pred)
knn_train_precision = precision_score(y_train, knn_train_pred, pos_label='Yes')
knn_train_recall = recall_score(y_train, knn_train_pred, pos_label='Yes')

knn_test_accuracy = accuracy_score(y_test, knn_test_pred)
knn_test_precision = precision_score(y_test, knn_test_pred, pos_label='Yes')
knn_test_recall = recall_score(y_test, knn_test_pred, pos_label='Yes')

# Evaluation metrics for decision tree classifier
tree_train_accuracy = accuracy_score(y_train, tree_train_pred)
tree_train_precision = precision_score(y_train, tree_train_pred, pos_label='Yes')
tree_train_recall = recall_score(y_train, tree_train_pred, pos_label='Yes')

tree_test_accuracy = accuracy_score(y_test, tree_test_pred)
tree_test_precision = precision_score(y_test, tree_test_pred, pos_label='Yes')
tree_test_recall = recall_score(y_test, tree_test_pred, pos_label='Yes')



print("Logistic Regression:")
print("Train Accuracy:", logistic_train_accuracy)
print("Train Precision:", logistic_train_precision)
print("Train Recall:", logistic_train_recall)
print("Test Accuracy:", logistic_test_accuracy)
print("Test Precision:", logistic_test_precision)
print("Test Recall:", logistic_test_recall)
print()

print("KNN Classifier:")
print("Train Accuracy:", knn_train_accuracy)
print("Train Precision:", knn_train_precision)
print("Train Recall:", knn_train_recall)
print("Test Accuracy:", knn_test_accuracy)
print("Test Precision:", knn_test_precision)
print("Test Recall:", knn_test_recall)
print()

print("Decision Tree Classifier:")
print("Train Accuracy:", tree_train_accuracy)
print("Train Precision:", tree_train_precision)
print("Train Recall:", tree_train_recall)
print("Test Accuracy:", tree_test_accuracy)
print("Test Precision:", tree_test_precision)
print("Test Recall:", tree_test_recall)

Logistic Regression:
Train Accuracy: 0.7912778904665314
Train Precision: 0.6514806378132119
Train Recall: 0.4416988416988417
Test Accuracy: 0.7941315664931378
Test Precision: 0.6843501326259946
Test Recall: 0.44947735191637633

KNN Classifier:
Train Accuracy: 0.8387423935091278
Train Precision: 0.7399232245681382
Train Recall: 0.5953667953667954
Test Accuracy: 0.7685754850922859
Test Precision: 0.587991718426501
Test Recall: 0.49477351916376305

Decision Tree Classifier:
Train Accuracy: 0.9908722109533469
Train Precision: 0.9921259842519685
Train Recall: 0.972972972972973
Test Accuracy: 0.7307146237576905
Test Precision: 0.5043782837127846
Test Recall: 0.5017421602787456


## ROUND 3

In [231]:
from sklearn.model_selection import cross_val_score

models = [
    ('Logistic Regression', logistic_model),
    ('KNN Classifier', knn_model),
    ('Decision Tree Classifier', tree_model)
]


# Applying k-fold cross-validation and check the model scores

for model_name, model in models:
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f'{model_name}:')
    print('Cross-Validation Scores:', scores)
    print('Mean Accuracy:', scores.mean())
    print('Standard Deviation:', scores.std())
    print()


Logistic Regression:
Cross-Validation Scores: [0.80020284 0.78904665 0.78498986 0.79208925 0.78803245]
Mean Accuracy: 0.7908722109533469
Standard Deviation: 0.005187307039571801

KNN Classifier:
Cross-Validation Scores: [0.78296146 0.77079108 0.77180527 0.77383367 0.76876268]
Mean Accuracy: 0.7736308316430021
Standard Deviation: 0.004943633917628382

Decision Tree Classifier:
Cross-Validation Scores: [0.70588235 0.72718053 0.71501014 0.69472617 0.72413793]
Mean Accuracy: 0.7133874239350912
Standard Deviation: 0.01194173308684383



## ROUND 4

In [232]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier() # Initializing the Random Forest Classifier
rf_model.fit(X_train_scaled, y_train) # Fit the model on the training data


# Making predictions on train and test sets
rf_train_pred = rf_model.predict(X_train_scaled)
rf_test_pred = rf_model.predict(X_test_scaled)


# Calculating accuracy for Random Forest Classifier
rf_train_accuracy = accuracy_score(y_train, rf_train_pred)
rf_test_accuracy = accuracy_score(y_test, rf_test_pred)




print("Random Forest Classifier:")
print("Train Accuracy:", rf_train_accuracy)
print("Test Accuracy:", rf_test_accuracy)

Random Forest Classifier:
Train Accuracy: 0.9908722109533469
Test Accuracy: 0.7657359204921912


In [235]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5) # Initialize GridSearchCV


grid_search.fit(X_train_scaled, y_train) # Fit GridSearchCV on the training data


# Get the best parameters and best score
best_params = grid_search.best_params_ 
best_score = grid_search.best_score_



print("Best Parameters:", best_params)
print("Best Score:", best_score)


Best Parameters: {'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 300}
Best Score: 0.7916835699797161


In [236]:
final_rf_model = RandomForestClassifier(**best_params) # Initialize the Random Forest Classifier with the best parameters

final_rf_model.fit(X_train_scaled, y_train) # Fit the final model on the training data

# Make predictions on train and test sets
final_rf_train_pred = final_rf_model.predict(X_train_scaled)
final_rf_test_pred = final_rf_model.predict(X_test_scaled)

# Calculate accuracy for the final model
final_rf_train_accuracy = accuracy_score(y_train, final_rf_train_pred)
final_rf_test_accuracy = accuracy_score(y_test, final_rf_test_pred)

# Print the accuracy for the final model
print("Final Random Forest Classifier:")
print("Train Accuracy:", final_rf_train_accuracy)
print("Test Accuracy:", final_rf_test_accuracy)


Final Random Forest Classifier:
Train Accuracy: 0.801419878296146
Test Accuracy: 0.7927117841930904


In [237]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Check for imbalance
class_counts = y_train.value_counts()
print("Class Counts:")
print(class_counts)

# Upsample using SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Downsampling using RandomUnderSampler
rus = RandomUnderSampler()
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

# Fit the model and check accuracy after resampling
model.fit(X_train_resampled, y_train_resampled)
train_pred = model.predict(X_train_resampled)
test_pred = model.predict(X_test_scaled)
train_accuracy = accuracy_score(y_train_resampled, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

# Print the accuracy after resampling
print("Accuracy after resampling:")
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Class Counts:
No     3635
Yes    1295
Name: Churn, dtype: int64
Accuracy after resampling:
Train Accuracy: 0.9918918918918919
Test Accuracy: 0.673450070989115


## MANAGING IMBALANCE IN THE DATASET

In [247]:
# Andy's function

def evaluate_model(note, model, X_test, y_test, results):
    pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    false_negatives = confusion_matrix(y_test, pred)[1][0]
    new_result = pd.DataFrame({'note':note,'accuracy':score,'precision':precision,'recall':recall,'f1_score':f1,'false_negatives':false_negatives},index=[0])
    return pd.concat([results,new_result],axis=0)